# This is Version 1 of our visualization tool
I use the following code to display feature lines (detected by CrestCODE based on curvature)

In [1]:
import os
import numpy as np
import open3d as o3d
import igl
import meshplot as mp

In [153]:
# extensions
PLY = ".ply"
TXT = ".txt"
# utility
NEIGHBOR = 6
MESHNAME = "StrongFeatureTester"
MESHFILE = "../Open3D_Test/" + MESHNAME + PLY
PSEUDO_PLY2 = "ply2" + TXT
NEWLINE = '\n'
MAX_neighbor = 6
CRESTLINE = 1 # 0 means NO Crest line, 1 means YES Crest line
# renaming CrestCode output!
RIDGES = "Ridge_" + "k=" + str(NEIGHBOR) + "_" + MESHNAME + TXT
RAVINES = "Ravine_" + "k=" + str(NEIGHBOR) + "_" + MESHNAME + TXT
OUTPUT_rename = "Output_" + "k=" + str(NEIGHBOR) + "_" + MESHNAME + TXT
PLY2_rename = "PLY2_" + "k=" + str(NEIGHBOR) + "_" + MESHNAME + TXT

In [128]:
"""Functions: for raw .ply input!"""
# input: mesh file (path)
# output1: vertices (N, 3) array; x y z coordinates
# output2: faces (F, 3) array, format: [a id, b id, c id]
def get_mesh(path):
    mesh = o3d.io.read_triangle_mesh(MESHFILE)
    print(mesh)
    verts = np.asarray(mesh.vertices)
    faces = np.asarray(mesh.triangles)
    return verts, faces
# input: output file name, neighbor (k), crestline (1 = yes, 0 = no), vertices (array), faces (array)
# does: txt file <- for CrestCODE
def to_pseudo_PLY2(name, neighbor, crestline, verts, faces):
    NEWLINE = '\n'
    verts_count = len(verts)
    faces_count = len(faces)
    with open(name, 'w') as f:
        f.write(str(verts_count) + NEWLINE)
        f.write(str(faces_count) + NEWLINE)
        f.write(str(neighbor) + NEWLINE)
        f.write(str(crestline) + NEWLINE)
        for i in range(verts_count): # all vertices x y z coordinates
            f.write(" ".join(map(str, verts[i])) + NEWLINE)
        for j in range(faces_count): # all faces a b c vertices ids
            f.write(" ".join(map(str, faces[j])) + NEWLINE)
    print(f"Success: {PSEUDO_PLY2}")

In [154]:
mesh_verts, mesh_faces = get_mesh(MESHFILE)
to_pseudo_PLY2(PSEUDO_PLY2, NEIGHBOR, CRESTLINE, mesh_verts, mesh_faces)

TriangleMesh with 32104 points and 63500 triangles.
Success: ply2.txt


# BOOM! You ran CrestCODE and got your .txt outputs!
<br>You ran command in the CrestCODE folder (through Terminal) and got .txt files for: output, ravines, ridges
>the input: ./setCurvature ply2.txt output.txt
><br>Note that 'ply2.txt' corresponds to (the value of) PSEUDO_PLY2

<br>The .txt show up in the CrestCODE folder
<br>You moved those files Back to Open3D_Test!

In [155]:
#cleaning up!
os.rename("ridges.txt", RIDGES)
os.rename("ravines.txt", RAVINES)
os.rename("output.txt", OUTPUT_rename)
os.rename("ply2.txt", PLY2_rename)

In [134]:
"""Functions: for CrestCODE output"""
# get data from Ridges.txt OR Ravines.txt
# input: meshfile name, mesh vertices, mesh faces
# output1: crest line vertices (V, 4) array, format: [x coordinate, y coordinate, z coordinate, connected component]
# output2: connected components (N, 3) array, format: [Ridgeness, Sphericalness, Cyclideness]
# output3: crest line edges (E, 3) array, format: [u vertex index, v vertex index, triangle ID]
def ReadCrestLine(filename):
    f = open(filename, 'r')
    V = int(f.readline())
    E = int(f.readline())
    N = int(f.readline()) # num of connected components;
    crestline_vertices = np.zeros(shape = (V, 4)) # crest line vertices
    for i in range(V): 
        line = f.readline()
        crestline_vertices[i] = [float(n) for n in line.split()]
    crestline_connected_cmp = np.zeros(shape = (N, 3)) # index = connected cmp ID
    for j in range(N):
        line = f.readline()
        crestline_connected_cmp[j] = [float(n) for n in line.split()]
    # edges (u,v): [vtx ID of u, vtx ID of v, triangle ID]
    crestline_edges = np.zeros(shape = (E, 3), dtype=int) # index = edge ID
    for k in range(E):
        line = f.readline()
        crestline_edges[k] = [n for n in line.split()]
    return V, crestline_vertices, E, crestline_edges, N, crestline_connected_cmp
# input1: crestline edges (E, 3) array, format: [u vertex index, v vertex index, triangle ID],
# input2: crestline vertices (V, 4) array, format: [x coordinate, y coordinate, z coordinate, connected component]
# output1: vertex U (E, 3) array, format: [x coordinate, y coordinate, z coordinate]
# output2: vertex V (E, 3) array, format: [x coordinate, y coordinate, z coordinate]
# edge E[i] = (U[i], V[i])
def getVertex_Pairs(crestline_edges, crestline_vertices):
    E = len(crestline_edges)
    V = len(crestline_vertices)
    U = np.zeros(shape=(E, 3))
    V = np.zeros(shape=(E, 3))
    U = np.zeros(shape = (E, 3))
    V = np.zeros(shape = (E, 3))
    for i in range(E): # use crestline_edges to find vertex index, only take first 3 numbers
        # if E[i] = a b c, this means that E[i] = (U[a], v[b])
        U[i] = crestline_vertices[crestline_edges[i][0]][0:3]
        V[i] = crestline_vertices[crestline_edges[i][1]][0:3]
    return U, V # can feed directly into meshplot :)
# input1: crestline edges (E, 3) array, format: [u vertex index, v vertex index, triangle ID],
# input2: faces frequency (F, max(k)) array, format: [face_id, frequency] <- by reference, exists outside this function
# input3: neighbor (k)
# does: add frequency values to: faces frequency array, in the column corresponding to k
def getTriangles(crestline_edges, faces_frequency, neighbor):
    # to find faces we want, we need to check edges!
    column = neighbor - 1
    E = len(crestline_edges)
    for e in range(E):
        face_id = crestline_edges[e][2]
        if(face_id != -1):
            faces_frequency[face_id][column] += 1    
# input: faces_frequency (F, max(k)) array, format: [face_id, frequency] <- by reference, exists outside this function
# output: faces_color_crestline (F, 3) array, format: [R, G, B], where each in [0, 1] float
def colorize_by_strength(faces_frequency):
    faces_frequency_total = np.sum(faces_frequency, axis=1)
    faces_color_crestline = np.ones(shape=(len(mesh_faces), 3))
    max_frequency = np.max(faces_frequency_total)
    for f in range(len(mesh_faces)):
        subtract = faces_frequency_total[f]/max_frequency
        faces_color_crestline[f][0] -= subtract
    return faces_color_crestline

## **Visualization**


In [135]:
"""run ONCE"""
faces_frequency = np.zeros(shape=(len(mesh_faces), 6))

In [156]:
V, crestline_vertices, E, crestline_edges, N, crestline_connected_cmp = ReadCrestLine(filename=RAVINES)
U, V = getVertex_Pairs(crestline_edges=crestline_edges, crestline_vertices=crestline_vertices)
getTriangles(crestline_edges=crestline_edges, faces_frequency=faces_frequency, neighbor=NEIGHBOR)
faces_color_crestline = colorize_by_strength(faces_frequency=faces_frequency)
p = mp.plot(v=mesh_verts, f=mesh_faces, c=faces_color_crestline)
# p.add_lines(U, V, shading={"line_color": "red", "line_width": 100.0})
# p.save("TEST_Ravine_" + MESHNAME)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.6551961…

In [175]:
p.save(filename="frequency_monotonic")

Plot saved to file frequency_monotonic.html.


In [187]:
# input1: maximum neighbor (k)
# input2: mesh_faces (F, 3) array
# output: faces_frequency (F, max(k)) array, 
#   format: faces_frequency[i][j] = 1 means ith face contains crestlines for neighbor = j; 0 otherwise
def read_ravine_multiple(max_neighbor, mesh_faces):
    faces_freq = np.zeros(shape=(len(mesh_faces), max_neighbor))
    for n in range(1, max_neighbor):
        filename = RAVINES = "Ravine_" + "k=" + str(n) + "_" + MESHNAME + TXT
        V, crestline_vertices, E, crestline_edges, N, crestline_connected_cmp = ReadCrestLine(filename=RAVINES)
        getTriangles(crestline_edges=crestline_edges, faces_frequency=faces_freq, neighbor=NEIGHBOR)
    return faces_freq
# input: faces_frequency (F, max(k)) array
# output: faces_colorful_crestline (F, 3) array, format: [R, G, B], where each in [0, 1] float
#   [small, 1, 1] = turquoise
#   [1, small, 1] = purple
#   [1, 1, small] = yellow
#   [1, small, small] = red
#   [small, small, 1] = blue
#   [small, 1, small] = green
colors = {
    '1': np.array([1.0, 0.1, 1.0]), # purple
    '2': np.array([0.1, 1.0, 0.1]), # turquoise
    '3': np.array([0.1, 0.1, 1.0]), # blue
    '4': np.array([0.1, 1.0, 0.1]), # green
    '5': np.array([1.0, 1.0, 0.1]), # yellow
    '6': np.array([1.0, 0.1, 0.1]), # red
    'threshold': np.array([0, 0, 0]) # black
    }
def colorful_by_strength(freq):
    tally = np.sum(freq, axis=1)
    faces_color = np.ones(shape=(len(freq), 3))
    for f in range(len(freq)):
        if(tally[f] == 6):
            faces_color[f] = colors['6']
        elif(tally[f] == 5):
            faces_color[f] = colors['5']
        elif(tally[f] == 4):
            faces_color[f] = colors['4']
        elif(tally[f] == 3):
            faces_color[f] = colors['3']
        elif(tally[f] == 2):
            faces_color[f] = colors['2']
        elif(tally[f] == 1):
            faces_color[f] = colors['1']
        else:
            faces_color[f] = np.ones(3)
    return faces_color
# input1: faces_frequency (F, max(k)) array
# input2: threshold
# output: strongest_color (F, 3) array, format: [R, G, B], where each in [0, 1] float
#   ONLY keep color of 
def colorful_by_strongest(freq_, threshold):
    tally = np.sum(freq_, axis=1)
    faces_color = np.ones(shape=(len(freq), 3))
    for f in range(len(freq)):
        if(tally[f] >= threshold):
            faces_color[f] = colors['threshold']
        else:
            faces_color[f] = np.ones(3)
    return faces_color
freq = read_ravine_multiple(max_neighbor=MAX_neighbor, mesh_faces=mesh_faces)
colorful = colorful_by_strength(freq=freq)
color_threshold = colorful_by_strongest(freq_=freq, threshold=5)
q = mp.plot(v=mesh_verts, f=mesh_faces, c=color_threshold)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.6551961…

In [188]:
q.save(filename="frequency_colorful_threshold=5")

Plot saved to file frequency_colorful_threshold=5.html.


In [200]:
"""#Example# get: faces; from: face index"""
F = np.asarray([0,1,2])
G = np.zeros(shape=(1,4))
G[:,0:3] = F
print(G)
# got: 
# [edge_u_index, edge_v_index, face_index] <- edges
# [a_index, b_index, c_index] @ face_index
# [x, y, z] @ a_index
# want:
# select faces by index
# tally appearance count for ea face

[[0. 1. 2. 0.]]


In [198]:
"""#Example# get: vertices; from: vertex index"""
A = np.asarray([[0, 1, 2]]) # edges, form: (u index, v index)
B = np.asarray([[5,6,0],[7,8,0],[9,10,0]]) # verts, form: (x, y, z)
# want: C = [5,6], D = [7,8] <- selected verts by indices; indices from edges
C = np.zeros(shape=(A.shape[0],2))
D = np.zeros(shape=(A.shape[0],2))
for i in range(len(A)):
    C[i] = B[A[i][0]][0:2]
    D[i] = B[A[i][1]][0:2]
print(B[1][0:2])
print(C)
print(D)

[7 8]
[[5. 6.]]
[[7. 8.]]


In [138]:
(V, N, E, CL_vertices, connected_cmp, edges) = RawData(RAVINES)
selected_faces = np.zeros((faces.shape[0],4))
selected_faces[:,0:3] = faces
selected_faces[1]



array([14., 13.,  3.,  0.])

In [135]:
triangle_indices = edges[:,2]
for i in range(len(triangle_indices)):
    if(i != -1):
        selected_faces[3] += 1
print(selected_faces[55498])

[28474. 28472. 24778.     0.]


In [ ]:
# show ravines (very nice)
ravines(RAVINES, verts, faces)

In [ ]:
# show entire mesh (no nothing)
p = mp.plot(verts, faces, return_plot=True)

In [52]:
# Save
p.save("FeatureLines_" + MESHNAME + "_neighbor=" + str(NEIGHBOR))

Plot saved to file FeatureLines_tbk_crop_230328_depth=12_correctNormals_crop_2_neighbor=6.html.


In [103]:
# Reference: 
# http://www.open3d.org/docs/0.11.1/tutorial/visualization/visualization.html
(V, N, E, vertices, connected_cmp, edges) = RawData(RAVINES)
points = vertices[:,:3]
lines = edges[:,:2]
colors = [[1, 0, 0] for i in range(E)]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)
o3d.io.write_line_set(RAVINES + PLY, line_set, print_progress=True)
#test

Writing PLY: [========================================] 100%


True

In [ ]:
# all feature lines
show(RIDGES, RAVINES, verts, faces)
# just show ridges (not very useful)
ridges(RIDGES, verts, faces)